# Dowload data

In [1]:
!gdown 1BVjrYwkQ69SPXhABMpaWCwVie2OPlXL6
!unzip -q data_word -d data


Downloading...
From (original): https://drive.google.com/uc?id=1BVjrYwkQ69SPXhABMpaWCwVie2OPlXL6
From (redirected): https://drive.google.com/uc?id=1BVjrYwkQ69SPXhABMpaWCwVie2OPlXL6&confirm=t&uuid=6c001899-e77d-4b65-bf14-ad91d10684e0
To: /kaggle/working/data_word.zip
100%|██████████████████████████████████████| 1.46G/1.46G [00:15<00:00, 93.7MB/s]


# Installation

In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2
!pip install jiwer
!pip install einops addict easydict


# Unsloth

In [3]:
#!pip install --upgrade --force-reinstall --no-cache-dir --no-deps unsloth unsloth_zoo transformers timm

from huggingface_hub import snapshot_download
snapshot_download("unsloth/DeepSeek-OCR", local_dir = "deepseek_ocr")

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

LICENSE: 0.00B [00:00, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

assets/show2.jpg:   0%|          | 0.00/216k [00:00<?, ?B/s]

...

processor_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

'/kaggle/working/deepseek_ocr'

In [4]:
import os, logging
os.environ["UNSLOTH_WARN_UNINITIALIZED"] = "0"

# Gỡ các handler dạng Unsloth khỏi logger transformers.modeling_utils
logger = logging.getLogger("transformers.modeling_utils")
new_handlers = []
for h in logger.handlers:
    # Unsloth thường đặt handler có class name đặc trưng, ta lọc theo tên
    if "Unsloth" in h.__class__.__name__:
        continue
    new_handlers.append(h)
logger.handlers = new_handlers


# Import

In [5]:
import os, json, random, argparse, unicodedata
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple

import torch
from jiwer import wer, cer
from huggingface_hub import snapshot_download

from unsloth import FastVisionModel, is_bf16_supported
from transformers import AutoModel, Trainer, TrainingArguments

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-13 12:21:18.243077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765628478.629207      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765628478.776146      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

🦥 Unsloth Zoo will now patch everything to make training faster!


# Fine-Tuning

In [6]:
# ========= (1) Text utils =========
def normalize_text(s: str) -> str:
    # Giữ dấu tiếng Việt ổn định (NFC), normalize whitespace nhẹ
    s = unicodedata.normalize("NFC", s or "")
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    s = " ".join(s.split())
    return s.strip()

def set_seed(seed: int):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

## Data Prep

In [7]:
# ========= (2) Data loader: scan .png/.txt pairs =========
def collect_pairs(root: str, recursive: bool = True) -> List[Dict[str, str]]:
    """
    Expect:
      <stem>.png
      <stem>.txt   (ground truth)
    Return list of {"image_path": ..., "text": ...}
    """
    items = []
    walker = os.walk(root) if recursive else [(root, [], os.listdir(root))]
    for d, _, files in walker:
        pngs = [f for f in files if f.lower().endswith(".png")]
        for png in pngs:
            stem = png[:-4]
            txt = stem + ".txt"
            img_path = os.path.join(d, png)
            txt_path = os.path.join(d, txt)
            if not os.path.exists(txt_path):
                continue
            with open(txt_path, "r", encoding="utf-8-sig") as f:
                gt = normalize_text(f.read())
            if gt == "":
                continue
            items.append({"image_path": img_path, "text": gt})
    return items

# ========= (3) Conversation format =========
def to_conversation(sample: Dict[str, str], instruction: str) -> Dict[str, Any]:
    # NOTE: để nhẹ RAM, ta truyền image_path (string) và collator sẽ tự Image.open
    return {
        "messages": [
            {"role": "<|User|>", "content": instruction, "images": [sample["image_path"]]},
            {"role": "<|Assistant|>", "content": sample["text"]},
        ]
    }


## Create datacollator

In [8]:
# ========= (4) Collator (dựa trên class bạn đang dùng, thêm hỗ trợ path string) =========
import math, io
from PIL import Image, ImageOps
from torch.nn.utils.rnn import pad_sequence
from deepseek_ocr.modeling_deepseekocr import text_encode, BasicImageTransform, dynamic_preprocess

class DeepSeekOCRDataCollator:
    def __init__(self, tokenizer, model, image_size=640, base_size=1024, crop_mode=True, train_on_responses_only=True):
        self.tokenizer = tokenizer
        self.model = model
        self.image_size = image_size
        self.base_size = base_size
        self.crop_mode = crop_mode
        self.image_token_id = 128815
        self.dtype = model.dtype
        self.train_on_responses_only = train_on_responses_only

        self.image_transform = BasicImageTransform(
            mean=(0.5, 0.5, 0.5),
            std=(0.5, 0.5, 0.5),
            normalize=True
        )
        self.patch_size = 16
        self.downsample_ratio = 4

        self.bos_id = tokenizer.bos_token_id if getattr(tokenizer, "bos_token_id", None) is not None else 0

    def deserialize_image(self, image_data) -> Image.Image:
        # ====== PATCH quan trọng cho dataset của bạn: support path string ======
        if isinstance(image_data, str):
            return Image.open(image_data).convert("RGB")
        if isinstance(image_data, Image.Image):
            return image_data.convert("RGB")
        if isinstance(image_data, dict):
            if "bytes" in image_data:
                image = Image.open(io.BytesIO(image_data["bytes"]))
                return image.convert("RGB")
            if "path" in image_data:
                return Image.open(image_data["path"]).convert("RGB")
        raise ValueError(f"Unsupported image format: {type(image_data)}")

    def process_image(self, image: Image.Image):
        images_list, images_crop_list, images_spatial_crop = [], [], []

        if self.crop_mode:
            if image.size[0] <= 640 and image.size[1] <= 640:
                crop_ratio = (1, 1)
                images_crop_raw = []
            else:
                images_crop_raw, crop_ratio = dynamic_preprocess(
                    image, min_num=2, max_num=9, image_size=self.image_size, use_thumbnail=False
                )

            global_view = ImageOps.pad(
                image, (self.base_size, self.base_size),
                color=tuple(int(x * 255) for x in self.image_transform.mean)
            )
            images_list.append(self.image_transform(global_view).to(self.dtype))

            width_crop_num, height_crop_num = crop_ratio
            images_spatial_crop.append([width_crop_num, height_crop_num])

            if width_crop_num > 1 or height_crop_num > 1:
                for crop_img in images_crop_raw:
                    images_crop_list.append(self.image_transform(crop_img).to(self.dtype))

            num_queries = math.ceil((self.image_size // self.patch_size) / self.downsample_ratio)
            num_queries_base = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)

            tokenized_image = ([self.image_token_id] * num_queries_base + [self.image_token_id]) * num_queries_base
            tokenized_image += [self.image_token_id]
            if width_crop_num > 1 or height_crop_num > 1:
                tokenized_image += ([self.image_token_id] * (num_queries * width_crop_num) + [self.image_token_id]) * (
                    num_queries * height_crop_num
                )
        else:
            crop_ratio = (1, 1)
            images_spatial_crop.append([1, 1])

            global_view = ImageOps.pad(
                image, (self.base_size, self.base_size),
                color=tuple(int(x * 255) for x in self.image_transform.mean)
            )
            images_list.append(self.image_transform(global_view).to(self.dtype))

            num_queries = math.ceil((self.base_size // self.patch_size) / self.downsample_ratio)
            tokenized_image = ([self.image_token_id] * num_queries + [self.image_token_id]) * num_queries
            tokenized_image += [self.image_token_id]

        return images_list, images_crop_list, images_spatial_crop, tokenized_image

    def process_single_sample(self, messages: List[Dict]) -> Dict[str, Any]:
        images = []
        for m in messages:
            for img in m.get("images", []) or []:
                images.append(self.deserialize_image(img))

        tokenized_str, images_seq_mask = [], []
        images_list, images_crop_list, images_spatial_crop = [], [], []
        prompt_token_count = -1
        assistant_started = False
        image_idx = 0

        tokenized_str.append(self.bos_id)
        images_seq_mask.append(False)

        for m in messages:
            role = m["role"]
            content = m["content"]

            if role == "<|Assistant|>":
                if not assistant_started:
                    prompt_token_count = len(tokenized_str)
                    assistant_started = True
                content = f"{content.strip()} {self.tokenizer.eos_token}"

            text_splits = content.split("<image>")
            for i, text_part in enumerate(text_splits):
                tok = text_encode(self.tokenizer, text_part, bos=False, eos=False)
                tokenized_str.extend(tok)
                images_seq_mask.extend([False] * len(tok))

                if i < len(text_splits) - 1:
                    img = images[image_idx]
                    il, cl, sc, tok_img = self.process_image(img)
                    images_list.extend(il)
                    images_crop_list.extend(cl)
                    images_spatial_crop.extend(sc)

                    tokenized_str.extend(tok_img)
                    images_seq_mask.extend([True] * len(tok_img))
                    image_idx += 1

        if not assistant_started:
            prompt_token_count = len(tokenized_str)

        images_ori = torch.stack(images_list, dim=0)
        images_spatial_crop_tensor = torch.tensor(images_spatial_crop, dtype=torch.long)
        images_crop = torch.stack(images_crop_list, dim=0) if images_crop_list else torch.zeros(
            (1, 3, self.base_size, self.base_size), dtype=self.dtype
        )

        return {
            "input_ids": torch.tensor(tokenized_str, dtype=torch.long),
            "images_seq_mask": torch.tensor(images_seq_mask, dtype=torch.bool),
            "images_ori": images_ori,
            "images_crop": images_crop,
            "images_spatial_crop": images_spatial_crop_tensor,
            "prompt_token_count": prompt_token_count,
        }

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        batch_data = [self.process_single_sample(f["messages"]) for f in features]

        input_ids_list = [x["input_ids"] for x in batch_data]
        images_seq_mask_list = [x["images_seq_mask"] for x in batch_data]
        prompt_counts = [x["prompt_token_count"] for x in batch_data]

        input_ids = pad_sequence(input_ids_list, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        images_seq_mask = pad_sequence(images_seq_mask_list, batch_first=True, padding_value=False)

        labels = input_ids.clone()
        labels[labels == self.tokenizer.pad_token_id] = -100
        labels[images_seq_mask] = -100
        for i, pc in enumerate(prompt_counts):
            if pc > 0:
                labels[i, :pc] = -100

        attention_mask = (input_ids != self.tokenizer.pad_token_id).long()
        images_batch = [(x["images_crop"], x["images_ori"]) for x in batch_data]
        images_spatial_crop = torch.cat([x["images_spatial_crop"] for x in batch_data], dim=0)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
            "images": images_batch,
            "images_seq_mask": images_seq_mask,
            "images_spatial_crop": images_spatial_crop,
        }


## Inference and eval

In [9]:
# ========= (5) Inference helper =========
@torch.inference_mode()
def ocr_one(model, tokenizer, image_path: str, prompt: str, base_size=1024, image_size=640, crop_mode=True) -> str:
    out = model.infer(
        tokenizer,
        prompt=prompt,
        image_file=image_path,
        output_path="outputs",
        base_size=base_size,
        image_size=image_size,
        crop_mode=crop_mode,
        save_results=False,
        test_compress=False,
    )
    # robust parse
    if isinstance(out, str):
        return normalize_text(out)
    if isinstance(out, list) and len(out) > 0:
        return normalize_text(str(out[0]))
    if isinstance(out, dict):
        for k in ["text", "result", "output"]:
            if k in out:
                return normalize_text(str(out[k]))
    return normalize_text(str(out))

def eval_loop(model, tokenizer, samples: List[Dict[str, str]], prompt: str,
              base_size=1024, image_size=640, crop_mode=True, max_eval=None) -> Dict[str, float]:
    if max_eval is not None:
        samples = samples[:max_eval]

    y_true, y_pred = [], []
    for s in samples:
        gt = normalize_text(s["text"])
        pr = ocr_one(model, tokenizer, s["image_path"], prompt, base_size, image_size, crop_mode)
        y_true.append(gt)
        y_pred.append(pr)

    return {
        "cer": float(cer(y_true, y_pred)),
        "wer": float(wer(y_true, y_pred)),
        "n": len(samples),
    }

## Main

In [10]:
# ========= (6) Main =========
def main1():
    ap = argparse.ArgumentParser()
    ap.add_argument("--data_dir", type=str, required=True)
    ap.add_argument("--out_dir", type=str, default="outputs/handwriting_lora")
    ap.add_argument("--val_ratio", type=float, default=0.05)
    ap.add_argument("--seed", type=int, default=3407)

    ap.add_argument("--max_steps", type=int, default=800)
    ap.add_argument("--lr", type=float, default=2e-4)
    ap.add_argument("--bsz", type=int, default=2)
    ap.add_argument("--gas", type=int, default=4)

    ap.add_argument("--image_size", type=int, default=640)
    ap.add_argument("--base_size", type=int, default=1024)
    ap.add_argument("--crop_mode", action="store_true")

    ap.add_argument("--max_eval", type=int, default=200)  # eval nhanh
    args = ap.parse_args()

    set_seed(args.seed)
    os.makedirs(args.out_dir, exist_ok=True)

    # Prompt: nên “cứng” và nhất quán cho chữ viết tay
    PROMPT = "<image>\nHãy chép lại nguyên văn chữ viết tay trong ảnh. Giữ đúng dấu tiếng Việt. "

    # 1) Download model snapshot
    #snapshot_download("unsloth/DeepSeek-OCR", local_dir="deepseek_ocr")

    # 2) Load base model
    model, tokenizer = FastVisionModel.from_pretrained(
        "unsloth/DeepSeek-OCR",
        load_in_4bit=False,
        auto_model=AutoModel,
        trust_remote_code=True,
        unsloth_force_compile=True,
        use_gradient_checkpointing="unsloth",
    )


    # 3) Load dataset
    import json, random

    pairs = collect_pairs(args.data_dir, recursive=True)
    rng = random.Random(42)
    rng.shuffle(pairs)

    n_val = max(1, int(len(pairs)*0.1))
    split = {"train": pairs[n_val:], "val": pairs[:n_val]}

    # Save train/val
    with open("split.json", "w", encoding="utf-8") as f:
        json.dump(split, f, ensure_ascii=False, indent=2)
    train_pairs = split["train"]
    val_pairs = split["val"]
    print(f"Total: {len(pairs)} | Train: {len(train_pairs)} | Val: {len(val_pairs)}")

    # 4) Baseline eval before finetune
    FastVisionModel.for_inference(model)
    base_metrics = eval_loop(
        model, tokenizer, val_pairs, PROMPT,
        base_size=args.base_size, image_size=args.image_size, crop_mode=args.crop_mode,
        max_eval=args.max_eval
    )
    print("Baseline:", base_metrics)

    # 5) Add LoRA (same target_modules)
    FastVisionModel.for_training(model)
    model = FastVisionModel.get_peft_model(
        model,
        target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
        r=16,
        lora_alpha=16,
        lora_dropout=0,
        bias="none",
        random_state=args.seed,
        use_rslora=False,
        loftq_config=None,
    )

    # 6) Convert to conversations
    train_conv = [to_conversation(s, PROMPT) for s in train_pairs]
    val_conv = [to_conversation(s, PROMPT) for s in val_pairs]

    # 7) Trainer
    collator = DeepSeekOCRDataCollator(
        tokenizer=tokenizer,
        model=model,
        image_size=args.image_size,
        base_size=args.base_size,
        crop_mode=args.crop_mode,
        train_on_responses_only=True,
    )

    targs = TrainingArguments(
        output_dir=args.out_dir,
        per_device_train_batch_size=args.bsz,
        gradient_accumulation_steps=args.gas,
        learning_rate=args.lr,
        warmup_steps=20,
        max_steps=args.max_steps,
        logging_steps=10,
        save_steps=200,
        optim="adamw_8bit",
        weight_decay=0.001,
        lr_scheduler_type="linear",
        seed=args.seed,
        fp16=not is_bf16_supported(),
        bf16=is_bf16_supported(),
        report_to="none",
        dataloader_num_workers=2,
        remove_unused_columns=False,  # quan trọng cho vision finetune
    )

    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        data_collator=collator,
        train_dataset=train_conv,
        args=targs,
    )

    trainer.train()

    # 8) Save LoRA adapters
    model.save_pretrained(os.path.join(args.out_dir, "lora_model"))
    tokenizer.save_pretrained(os.path.join(args.out_dir, "lora_model"))

    # 9) Eval after finetune
    FastVisionModel.for_inference(model)
    ft_metrics = eval_loop(
        model, tokenizer, val_pairs, PROMPT,
        base_size=args.base_size, image_size=args.image_size, crop_mode=args.crop_mode,
        max_eval=args.max_eval
    )
    print("Finetuned:", ft_metrics)

    # 10) Write report
    report = {"baseline": base_metrics, "finetuned": ft_metrics, "train_size": len(train_pairs), "val_size": len(val_pairs)}
    with open(os.path.join(args.out_dir, "eval_report.json"), "w", encoding="utf-8") as f:
        json.dump(report, f, ensure_ascii=False, indent=2)


# Run

In [ ]:
if __name__ == "__main__":
  import sys
  DATA_DIR = "/kaggle/working/data/InkData_word_processed"
  sys.argv = ["train_handwriting.py",
              "--data_dir", DATA_DIR,
              "--out_dir", "outputs/handwriting_lora",
              "--max_steps", "1500",
              "--crop_mode"]
  main1()

A new version of the following files was downloaded from https://huggingface.co/unsloth/DeepSeek-OCR:
- deepencoder.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/unsloth/DeepSeek-OCR:
- conversation.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/unsloth/DeepSeek-OCR:
- modeling_deepseekocr.py
- deepencoder.py
- conversation.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and 

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.12.5: Fast Deepseekocr patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.
You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-000001.safetensors:   0%|          | 0.00/6.67G [00:00<?, ?B/s]

Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at unsloth/DeepSeek-OCR and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

In [ ]:
# Inference test

import os, json, argparse, unicodedata
import torch
from huggingface_hub import snapshot_download
from unsloth import FastVisionModel
from transformers import AutoModel

def normalize_text(s: str) -> str:
    s = unicodedata.normalize("NFC", s or "")
    s = " ".join(s.split())
    return s.strip()

@torch.inference_mode()
def main2():
    ap = argparse.ArgumentParser()
    ap.add_argument("--image_dir", required=True)
    ap.add_argument("--lora_dir", required=True)
    ap.add_argument("--out_json", default="preds.json")
    ap.add_argument("--base_size", type=int, default=1024)
    ap.add_argument("--image_size", type=int, default=640)
    ap.add_argument("--crop_mode", action="store_true")
    args = ap.parse_args()

    PROMPT = "<image>\nHãy chép lại nguyên văn chữ viết tay trong ảnh. Giữ đúng dấu tiếng Việt. "

    snapshot_download("unsloth/DeepSeek-OCR", local_dir="deepseek_ocr_base")
    model, tokenizer = FastVisionModel.from_pretrained(
        args.lora_dir,
        load_in_4bit=False,
        auto_model=AutoModel,
        trust_remote_code=True,
        unsloth_force_compile=True,
        use_gradient_checkpointing="unsloth",
    )
    FastVisionModel.for_inference(model)

    results = {}
    for f in sorted(os.listdir(args.image_dir)):
        if not f.lower().endswith(".png"):
            continue
        p = os.path.join(args.image_dir, f)
        out = model.infer(tokenizer, prompt=PROMPT, image_file=p, output_path="outputs",
                          base_size=args.base_size, image_size=args.image_size, crop_mode=args.crop_mode,
                          save_results=False, test_compress=False)
        results[f] = normalize_text(out if isinstance(out, str) else str(out))

    with open(args.out_json, "w", encoding="utf-8") as fp:
        json.dump(results, fp, ensure_ascii=False, indent=2)


In [ ]:
if __name__ == "__main__":
  import sys
  INF_DIR = "/kaggle/working/data/InkData_word_processed/20140603_0003_BCCTC_tg_0_0_0.png"
  sys.argv = ["infer_handwriting.py",
              "--image_dir", INF_DIR,
              "--lora_dir", "lora_model",
              "--base_size",
              "--image_size",
              "--crop_mode"]
  main2()
